## Import historical data to MongoDB

#### Imports and basic configuration

In [24]:
import csv
import logging
import pymongo
import os
import codecs
import threading

logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)

#### MongoDB setup

In [25]:
client = pymongo.MongoClient('localhost', 27017)

In [26]:
if input("Sicher, dass du alles löschen willst?") in ["y","Y","j","J"]:
    try:
        client.drop_database("tanken")
    except:
        pass

In [27]:
db = client.tanken
preise = db.preise
stationen = db.stationen

#### Helper functions

In [28]:
inserted_uuids = set()

def load_csv(path):
    with codecs.open(path,"r", "utf-8") as file:
        return [row for row in csv.DictReader(file)]

def convert_types_prices(data):
    for d in data:
        try:
            for key in list(d.keys()):
                if key=="diesel":
                    d["diesel"] = float(d.get("diesel"))
                elif key=="e5":
                    d["e5"] = float(d.get("e5"))
                elif key=="e10":
                    d["e10"] = float(d.get("e10"))
                elif key=="dieselchange":
                    d["dieselchange"] = bool(d.get("dieselchange"))
                elif key=="e10change":
                    d["e10change"] = bool(d.get("e10change"))
                elif key=="e5change":
                    d["e5change"] = bool(d.get("e5change"))
        except Exception as e:
            logging.error("Cant convert data")

def load_and_insert_prices_csv(path, collection):
    logging.info("Loading "+ path)
    data = load_csv(path)
    convert_types_prices(data)
    logging.info("Loaded "+ path+ "! Inserting...")
    collection.insert_many(data)
    logging.info("Inserted!")

def convert_types_stations(data):
    pass

def load_and_insert_stations_csv(path, collection):
    logging.info("Loading "+ path)
    data = load_csv(path)
    convert_types_stations(data)
    cleared_data= list()
    for d in data:
        if d["uuid"] not in inserted_uuids:
            cleared_data.append(d)
            inserted_uuids.add(d["uuid"])
    logging.info("Loaded "+ path+ "! Inserting...")
    if len(cleared_data)>0:
        collection.insert_many(cleared_data)
    logging.info("Inserted!")


def import_dataset(name, collection, function):
    daten_dir = "daten/"

    if not os.path.exists(daten_dir):
        logging.error("Daten-Ordner nicht gefunden")
    else:
        if name in os.listdir(daten_dir):
            logging.info("Started import of "+name)
            for year in os.listdir(daten_dir+name+"/"):
                logging.info(name +" -> year:"+ year)
                for month in os.listdir(daten_dir+name+"/"+year):
                    logging.info(name +" -> month:"+ month)
                    for day in os.listdir(daten_dir+name+"/"+year+"/"+month):
                        function(daten_dir+name+"/"+year+"/"+month+"/"+day, collection)
    logging.info("Finished import of "+name)

#### Import execution

In [ ]:
#import_dataset("preise",preise,load_and_insert_prices_csv)
import_dataset("stationen",stationen,load_and_insert_stations_csv)

INFO: Started import of preise
INFO: preise -> year:2020
INFO: preise -> month:01
INFO: Loading daten/preise/2020/01/2020-01-01-prices.csv
INFO: Loaded daten/preise/2020/01/2020-01-01-prices.csv! Inserting...
INFO: Inserted!
INFO: Loading daten/preise/2020/01/2020-01-02-prices.csv
INFO: Loaded daten/preise/2020/01/2020-01-02-prices.csv! Inserting...
INFO: Inserted!
INFO: Loading daten/preise/2020/01/2020-01-03-prices.csv
INFO: Loaded daten/preise/2020/01/2020-01-03-prices.csv! Inserting...
INFO: Inserted!
INFO: Loading daten/preise/2020/01/2020-01-04-prices.csv
INFO: Loaded daten/preise/2020/01/2020-01-04-prices.csv! Inserting...
INFO: Inserted!
INFO: Loading daten/preise/2020/01/2020-01-05-prices.csv
INFO: Loaded daten/preise/2020/01/2020-01-05-prices.csv! Inserting...
INFO: Inserted!
INFO: Loading daten/preise/2020/01/2020-01-06-prices.csv
INFO: Loaded daten/preise/2020/01/2020-01-06-prices.csv! Inserting...
INFO: Inserted!
INFO: Loading daten/preise/2020/01/2020-01-07-prices.csv
INF

#### Analyse imported data
Find duplicates in stations collection:

In [ ]:
for i in stationen.aggregate([{"$group": {"_id": "$uuid","count": {"$sum": 1}}}]):
    if i["count"]!=1:
        print(i)

In [16]:
print(preise.find_one({"station_uuid":"123456"}))

{'_id': ObjectId('5fa860d91c4f7224e168c202'), 'date': '2020-01-01 00:00:04+01', 'station_uuid': '123456', 'diesel': '1.319', 'e5': '1.469', 'e10': '1.449', 'dieselchange': '1', 'e5change': '0', 'e10change': '0'}


In [23]:
d = load_csv("daten/preise/2020/01/2020-01-01-prices.csv")
convert_types_prices(d)
print(d[0])

{'date': '2020-01-01 00:00:04+01', 'station_uuid': '81d72fa3-5270-45af-816f-ba0abb7776f4', 'diesel': 1.319, 'e5': 1.469, 'e10': 1.449, 'dieselchange': True, 'e5change': True, 'e10change': True}
